In [27]:
import pickle
import pandas as pd
import numpy as np

with open('customer_orders.pkl', 'rb') as f:
    data = pickle.load(f)

print(f"{type(data)}")
print(f"num of records: {len(data)}")
print(data[0])

<class 'list'>
num of records: 50
{'id': 1, 'name': 'Customer 1', 'registration_date': '2022-12-31 04:19:19', 'orders': [{'order_id': 84, 'order_date': '2024-02-27 21:24:16', 'shipping_address': '840 Main St, City 8', 'order_total_value': 0, 'items': []}, {'order_id': None, 'order_date': '2025-05-03 08:09:20', 'shipping_address': '533 Main St, City 9', 'order_total_value': 2765.8, 'items': [{'item_id': 1, 'product_name': 'Item 1 for Order 2', 'category': 4, 'price': 52.7, 'quantity': 2}, {'item_id': 2, 'product_name': 'Item 2 for Order 2', 'category': 5, 'price': 498.45, 'quantity': 5}, {'item_id': 3, 'product_name': 'Item 3 for Order 2', 'category': 5, 'price': 168.15, 'quantity': 1}]}, {'order_id': 3, 'order_date': '2023-09-06 00:42:50', 'shipping_address': '588 Main St, City 3', 'order_total_value': 0, 'items': [{'item_id': 1, 'product_name': 'Item 1 for Order 3', 'category': 1, 'price': '$377.96', 'quantity': 1}]}]}


In [28]:
with open('vip_customers.txt', 'r') as f:
    vip_content = f.read()
print(f"{vip_content}")

35
11
1
17
23
43
4
15
8
37



In [30]:
no_items = 0
for customer in data:
    if 'orders' in customer:
        for order in customer['orders']:
            if 'items' in order and not order['items']:
                no_items += 1
print(f"{no_items}")

19


In [48]:

import pandas as pd
import pickle
import re
from typing import List, Dict, Set, Any
from datetime import datetime

class CustomerDataExtractor:
    def __init__(self, pickle_file: str, vip_file: str):
        self.pickle_file = pickle_file
        self.vip_file = vip_file
        self.category_mapping = {1: 'Electronics', 2: 'Apparel', 3: 'Books', 4: 'Home Goods'}
        self.raw_data: List[Dict] = []
        self.vip_customers: Set[int] = set()

    def _load_customer_data(self) -> None:
        """Load pickled customer data"""
        with open(self.pickle_file, 'rb') as f:
            self.raw_data = pickle.load(f)

    def _load_vip_customers(self) -> None:
        """Load VIP customer IDs from text file"""
        with open(self.vip_file, 'r') as f:
            self.vip_customers = {int(line.strip()) for line in f if line.strip()}

    def _clean_price(self, price: Any) -> float:
        """Convert price to float, handling $ prefix and other edge cases"""
        if price is None:
            return 0.0
        if isinstance(price, (int, float)):
            return float(price)
        if isinstance(price, str):
            # Strip $, commas, and any whitespace, then convert
            cleaned = price.replace('$', '').replace(',', '').strip()
            if cleaned == '':
                return 0.0
            try:
                return float(cleaned)
            except ValueError:
                return 0.0
        return 0.0  # fallback for other types

    def _clean_quantity(self, quantity: Any) -> int:
        """Convert quantity to int, handling edge cases"""
        if quantity is None:
            return 0
        if isinstance(quantity, (int, float)):
            return int(quantity)
        if isinstance(quantity, str):
            try:
                return int(float(quantity.strip()))  # handle "5.0" -> 5
            except ValueError:
                return 0
        return 0  # fallback

    def _clean_order_id(self, order_id: Any) -> int:
        """Convert order_id to int, handling string formats like 'ORD14'"""
        if order_id is None:
            return 0
        if isinstance(order_id, (int, float)):
            return int(order_id)
        if isinstance(order_id, str):
            # Extract numbers from string like 'ORD14' -> 14
            numbers = re.findall(r'\d+', order_id)
            if numbers:
                return int(numbers[0])
            return 0
        return 0

    def _flatten_data(self) -> List[Dict]:
        """Transform nested data into flat records"""
        flattened_records = []

        for customer in self.raw_data:
            try:
                customer_id = customer['id']
                customer_name = customer['name']
                registration_date = pd.to_datetime(customer['registration_date'], errors='coerce')
                is_vip = customer_id in self.vip_customers

                # Skip if registration date is invalid
                if pd.isna(registration_date):
                    continue

                # Handle customers without 'orders' key
                orders = customer.get('orders', [])

                for order in orders:
                    try:
                        # Skip orders with empty items OR None order_id
                        if not order.get('items'):
                            continue

                        order_id = self._clean_order_id(order.get('order_id'))
                        if order_id == 0:  # Skip if order_id couldn't be cleaned
                            continue

                        order_date = pd.to_datetime(order['order_date'], errors='coerce')

                        # Skip if order date is invalid
                        if pd.isna(order_date):
                            continue

                        # Calculate total order value for percentage calculation
                        order_total = sum(
                            self._clean_price(item.get('price', 0)) * self._clean_quantity(item.get('quantity', 0))
                            for item in order['items']
                        )

                        for item in order['items']:
                            try:
                                # Skip items with missing item_id
                                item_id = self._clean_order_id(item.get('item_id'))  # Reuse same cleaning logic
                                if item_id == 0:
                                    continue

                                unit_price = self._clean_price(item.get('price', 0))
                                quantity = self._clean_quantity(item.get('quantity', 0))
                                total_item_price = unit_price * quantity

                                # Handle percentage calculation
                                total_order_value_percentage = (
                                    (total_item_price / order_total * 100) if order_total > 0 else 0.0
                                )

                                record = {
                                    'customer_id': customer_id,
                                    'customer_name': customer_name,
                                    'registration_date': registration_date,
                                    'is_vip': is_vip,
                                    'order_id': order_id,
                                    'order_date': order_date,
                                    'product_id': item_id,
                                    'product_name': item.get('product_name', ''),
                                    'category': self.category_mapping.get(item.get('category'), 'Misc'),
                                    'unit_price': unit_price,
                                    'item_quantity': quantity,
                                    'total_item_price': total_item_price,
                                    'total_order_value_percentage': total_order_value_percentage
                                }

                                flattened_records.append(record)
                            except (KeyError, ValueError, TypeError):
                                continue
                    except (KeyError, ValueError, TypeError):
                        continue
            except (KeyError, ValueError, TypeError):
                continue

        return flattened_records

    def extract_to_dataframe(self) -> pd.DataFrame:
        """Main method: extract and transform data to DataFrame"""
        # Load data
        self._load_customer_data()
        self._load_vip_customers()

        # Flatten and create DataFrame
        flattened_data = self._flatten_data()
        df = pd.DataFrame(flattened_data)

        if df.empty:
            # Return empty DataFrame with correct columns if no data
            columns = ['customer_id', 'customer_name', 'registration_date', 'is_vip',
                      'order_id', 'order_date', 'product_id', 'product_name', 'category',
                      'unit_price', 'item_quantity', 'total_item_price', 'total_order_value_percentage']
            return pd.DataFrame(columns=columns)

        # Reorder columns to match spec exactly
        df = df[[
            'customer_id',
            'customer_name',
            'registration_date',
            'is_vip',
            'order_id',
            'order_date',
            'product_id',
            'product_name',
            'category',
            'unit_price',
            'item_quantity',
            'total_item_price',
            'total_order_value_percentage'
        ]]

        # Ensure data types match spec exactly
        df = df.astype({
            'customer_id': 'int',
            'customer_name': 'str',
            'registration_date': 'datetime64[ns]',
            'is_vip': 'bool',
            'order_id': 'int',
            'order_date': 'datetime64[ns]',
            'product_id': 'int',
            'product_name': 'str',
            'category': 'str',
            'unit_price': 'float',
            'item_quantity': 'int',
            'total_item_price': 'float',
            'total_order_value_percentage': 'float'
        })

        # Sort as required: customer_id, then order_id, then product_id (ascending)
        df = df.sort_values(['customer_id', 'order_id', 'product_id']).reset_index(drop=True)

        return df

In [49]:
extractor = CustomerDataExtractor('customer_orders.pkl', 'vip_customers.txt')
df = extractor.extract_to_dataframe()

print(f"DataFrame shape: {df.shape}")
print(f"Data types:\n{df.dtypes}")
print(f"\nFirst few rows:\n{df.head()}")

# Check some key things
print(f"\nUnique customers: {df['customer_id'].nunique()}")
print(f"VIP customers in data: {df['is_vip'].sum()}")
print(f"Categories found: {df['category'].unique()}")

# Save to CSV as required
df.to_csv('customer_order_analysis.csv', index=False)
print(f"\nSaved to customer_order_analysis.csv")

DataFrame shape: (162, 13)
Data types:
customer_id                              int64
customer_name                           object
registration_date               datetime64[ns]
is_vip                                    bool
order_id                                 int64
order_date                      datetime64[ns]
product_id                               int64
product_name                            object
category                                object
unit_price                             float64
item_quantity                            int64
total_item_price                       float64
total_order_value_percentage           float64
dtype: object

First few rows:
   customer_id customer_name   registration_date  is_vip  order_id  \
0            1    Customer 1 2022-12-31 04:19:19    True         3   
1            2    Customer 2 2022-05-27 00:23:28   False         5   
2            2    Customer 2 2022-05-27 00:23:28   False         5   
3            2    Customer 2 2022-05-27